In [1]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW
import torch.nn.functional as F

In [2]:
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
tokenizer.pad_token = tokenizer.eos_token

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [4]:
checkpoint = torch.load('cocktail_bot.pt', map_location='cpu')
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [5]:
# model = model.to(device)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2):

In [5]:
recipe_start = "TITLE:"
recipe_start = tokenizer(recipe_start, return_tensors= 'pt')
# recipe_start = recipe_start.to(device)

In [6]:
tokenizer.eos_token_id

50256

In [18]:
model.generate(**recipe_start, max_length=150, top_k=100, top_p=0.9, pad_token_id=tokenizer.eos_token_id, do_sample=True).tolist()[0]

[49560,
 2538,
 25,
 220,
 12419,
 4061,
 18973,
 56,
 327,
 11290,
 5603,
 4146,
 198,
 30910,
 40165,
 25,
 37401,
 11,
 340,
 447,
 247,
 82,
 257,
 447,
 247,
 82,
 4836,
 7958,
 640,
 13,
 3497,
 257,
 922,
 198,
 2032,
 3757,
 351,
 262,
 24554,
 1017,
 3889,
 290,
 262,
 41118,
 743,
 423,
 257,
 1738,
 284,
 198,
 42138,
 13,
 198,
 198,
 16,
 14,
 17,
 25799,
 716,
 533,
 33955,
 198,
 16,
 14,
 17,
 25799,
 5894,
 3326,
 14775,
 198,
 16,
 14,
 17,
 25799,
 30777,
 34711,
 13135,
 198,
 17931,
 23988,
 11053,
 25,
 220,
 27845,
 24554,
 427,
 3110,
 351,
 4771,
 13,
 198,
 3060,
 716,
 533,
 33955,
 11,
 3326,
 14775,
 11,
 30777,
 34711,
 13135,
 11,
 290,
 39733,
 13,
 198,
 41113,
 13,
 198,
 520,
 3201,
 656,
 257,
 24554,
 5405,
 13,
 50256]

In [12]:
print(tokenizer.decode([49560,  2538,    25,   220,  3001,  1503,  2767, 10468,   327, 11290,
          5603,  4146,   198, 30910, 40165,    25,   770,   220,  6833,   220,
         24554,   220,   318,   220,  1336,   220,   286,   220,  7758,   261,
           220,   290,   220, 47565,   220, 35919,    11,   198,  4480, 20269,
           286, 10912, 31738,   290, 18873,    13,   198,   198, 33018,   501,
           286,   352,    14,    17, 18873,   198,    17, 24405, 39733,   198,
            16,    14,    17, 25799, 17236,   324,   500,   198,    17, 24405,
          1090,   330,  5488,   198, 17931, 23988, 11053,    25,   220, 27845,
         24554,   427,  3110,   351,  4771,    13,   198,  3060, 18873, 13135,
            11, 39733,    11, 17236,   324,   500,    11,  1090,   330,  5488,
            11,   290,   198,   198,    79,   322,    67,  1068,  7543,    13,
           628, 41113,    13,   198,   520,  3201,   656,   257, 45550, 11277,
          5362,  5405,    13, 50256]))

TITLE:  AMARETTO COCKTAIL
DESCRIPTION: This  classic  cocktail  is  full  of  melon  and  peach  juices,
with hints of orange peel and lemon.

Juice of 1/2 lemon
2 ounces gin
1/2 ounce grenadine
2 ounces curacao
DIRECTIONS:  Fill cocktail shaker with ice.
 Add lemon juice, gin, grenadine, curacao, and

powdered sugar.

 Shake.
 Strain into a chilled martini glass.<|endoftext|>


# Alternative method to generating cocktails using logits

In [13]:
import numpy as np
def choose_from_top(probs, n=5):
    ind = np.argpartition(probs, -n)[-n:]
    top_prob = probs[ind]
    top_prob = top_prob / np.sum(top_prob) # Normalize
    choice = np.random.choice(n, 1, p = top_prob)
    token_id = ind[choice][0]
    return int(token_id)

In [16]:
def generate_recipe():
    prompt = tokenizer.encode("TITLE:", return_tensors='pt')[0]
    eos = False
    counter = 0
    while not eos:
        counter +=1
        print(counter)
        x = model(prompt.unsqueeze(0).unsqueeze(0), labels=prompt.unsqueeze(0).unsqueeze(0))[1]
        probs = torch.softmax(x[0,-1], dim=-1)[-1]
        next_word = torch.tensor([choose_from_top(probs.detach().numpy())])
        prompt = torch.cat([prompt.squeeze(), next_word])
        tokenizer.decode(next_word)
        if next_word.item() == 50256:
            eos = True
    print(tokenizer.decode(prompt))

In [15]:
generate_recipe()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
TITLE:  BULL’S HEAD
DESCRIPTION: This one is for when the’s’t up.

1 1/2 ounces light rum
1/2 ounce triple sec
1/2 ounce lime juice 
DIRECTIONS:  Fill cocktail shaker with ice.
 Add rum, triple sec, lime juice, and gin.
 Shake.
 Strain into a cocktail glass.<|endoftext|>
